#Validation notebook to perform the basic file level validations

1.Null checks
2.Schema checks
3.Column checks
4.dependency checks (if needed)
5.Unique constraint check (if needed)


In [1]:
obj_type = 'test'
file_type = 'parquet'
file_path = 'test'
obj_name = 'test'
lakehouse_name = 'test'
workspace_name = 'test'
unique_flg = 'N'
null_flg = 'N'
dep_flg = 'N'
unique_col = "testcol"
dccol1 = "testcol"
dccol2 = "testcol"
sourcename ='cse'
# obj_type = 'file'
# file_type = 'csv'
# file_path = 'Files/Input'
# obj_name = 'circuits_data_quality'
# lakehouse_name = 'Config_Lakehouse'
# workspace_name = 'CRM_WS_Dev'
# unique_flg = 'Y'
# null_flg = 'Y'
# dep_flg = 'Y'
# unique_col = "circuitId"
# dccol1 = "circuitRef"
# dccol2 = "name"

StatementMeta(, 762427d7-705b-4b3b-ac28-1cf84fa76f02, 3, Finished, Available, Finished)

In [ ]:
    SourceName = "sourceone"
    obj_name= "customers"
    LandingLayerPath= "abfss://data-sit@dlsuksdataplatformdev.dfs.core.windows.net/bronze/landing/sourceone/customers/"
    CertifiedRawLayerPath= "abfss://data-sit@dlsuksdataplatformdev.dfs.core.windows.net/bronze/certified/sourceone/raw/customers/"
    CertifiedStandardizedLayerPath= "abfss://data-sit@dlsuksdataplatformdev.dfs.core.windows.net/bronze/certified/sourceone/standardized/customers/"
    ErrorLayerPath= "abfss://data-sit@dlsuksdataplatformdev.dfs.core.windows.net/bronze/error/sourceone/customers/"
    BusinessRunTimeStamp = '20210913'
    IsFolderPath = 'Y'
    file_type = 'csv'
    Delim = ','
    SourceFeedEscape = ''
    SourceFeedHasHeader = 'Y'
    DBJDBCKVName = 'ControlDBJDBCConntring'
    KeyVaultName = 'kv-uks-dataplatform-dev'
    KeyVaultLinkedService = 'ls_azdp_kv'

In [2]:
%run data_loading_metadata_driven_framework_latest

StatementMeta(, 762427d7-705b-4b3b-ac28-1cf84fa76f02, 4, Finished, Available, Finished)

In [4]:
dq = metadata_ingestion_new()
if obj_type == 'file':
    if file_type == 'csv':
        df = spark.read.option("header", "True").option("delimiter", ",").option("inferSchema", True).csv(f"abfss://{workspace_name}@onelake.dfs.fabric.microsoft.com/{lakehouse_name}.Lakehouse/{file_path}/{obj_name}.{file_type}")
    elif file_type == 'parquet':
        df = spark.read.parquet(f"abfss://{workspace_name}@onelake.dfs.fabric.microsoft.com/{lakehouse_name}.Lakehouse/{file_path}/{obj_name}.{file_type}")
elif obj_type == 'table':
    df = spark.read.format("delta").load(f"abfss://{workspace_name}@onelake.dfs.fabric.microsoft.com/{lakehouse_name}.Lakehouse/Tables/{obj_name}")
else:
    raise RuntimeError("not a valid object type")

notebook_name = 'Data_Quality_class_calling_Notebook_%s'%(workspace_name)
input_object_name = obj_name
error_count = dq.data_quality(notebook_name, df, input_object_name, unique_flg, null_flg, dep_flg, unique_col, dccol1, dccol2)







StatementMeta(, 762427d7-705b-4b3b-ac28-1cf84fa76f02, 6, Finished, Available, Finished)

error count is 2


2

ExitValue: 2

In [ ]:
param=spark.sql(f""" select count(*) from dbo.giEntityDefinitionMapping where targetentityfeedname ='{input_object_name}' """)
colmn_check= schema_check(df,param)


In [ ]:
# Execute validation
def execute_validation(SourceName, obj_name, file_type, Delim, filename):
    # Initialize config, error, and warning dictionaries
    config, error, warn, target_path = {}, {}, {}, ""

    # Get config file path
    configFileName = filename.split(".")[0]

    # Fetch global config
    try:
        config = getValidationConfigFromDBforSourceFeed(SourceName, obj_name, file_type, Delim)
    except Exception as e:
        raise Exception("Config Error: " + str(e))

    print("validation_check------------")

    # Determine file path
    file_path = root_landing if IsFolderPath == 'N' else root_landing + "/" + filename
    print(file_path)

    # Read data
    df = read_source(file_path, config)

    # Call validations in config
    for function, param in config['validation_check'].items():
        try:
            print(f"calling function {function}")
            print(f"Parms: {param}")
            globals()[function](df, param)
        except Exception as e:
            print(f"Exception caught: {e}")
            if function in config["optional_check"]:
                warn[function] = e
            else:
                error[function] = e

    if error:
        target_path = root_error + "/" + filename
        move_file(spark, file_path, target_path)
        log_error(root_error, "/error.json", str(error))

        if warn:
            log_error(root_error, "warn.json", str(warn))

        print("target_path =", target_path)
        delete_file(file_path)
        raise Exception("Error in file. Check error.json for more info")
    else:
        rawFilePath = root_entity_certified_raw + filename
        move_file(spark, file_path, rawFilePath)

        standardizedFilePath = root_entity_certified_stadardized + "/" + filename
        config['format'] = "parquet"
        write_data(df, standardizedFilePath, config)
        target_path = root_entity_certified_raw

        if warn:
            warnFileName = filename + "_warn.json"
            log_error(target_path, warnFileName, str(warn))

        print("target_path =", target_path)
        delete_file(file_path)

    return error, target_path


#schema checks, column counts checks and allowable value checks

In [ ]:
try:
    """
    This function checks if the columns specified in 'param' are present and not NULL or blank in the dataframe.
    """
    def schema_check(df, param):
        missing_columns = [col for col in param if col not in df.columns]
        if missing_columns:
            raise Exception(f"The columns {missing_columns} are missing from the file")
        print("schema_check succeeded")

    """
    This function checks for allowed values in particular columns.
    """
    def known_value_check(df, param):
        for colname, values in param.items():
            filter_string = " and ".join([f"{colname} != '{value}'" for value in values])
            df_filtered = df.filter(filter_string)
            if df_filtered.limit(1).count() != 0:
                raise Exception(f"Unknown values other than the accepted values found for {colname}")
        print("known value check succeeded")

except Exception as e:
    el.log_error(notebook, e)


#get the copied file details from bronze folder to be validated

In [ ]:
obj_name += '.csv'

try:
    filenames = [filename.name for filename in mssparkutils.fs.ls(file_path + obj_name)]
    if obj_name in filenames:
        print(obj_name, obj_name, obj_name)
        error, target_path = execute_validation(SourceName, obj_name, file_type, Delim, obj_name)
    else:
        print('Give the input file details')
except Exception as e:
    mssparkutils.notebook.exit(ex)


In [4]:
%%sql
SELECT * FROM cs_security_lh.dbo.loadcontroltable LIMIT 1000


StatementMeta(, 8dd3a5dc-5f8f-4a13-897e-dd4edfc4d97b, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 14 fields>

In [3]:
df = spark.sql("SELECT * FROM cs_security_lh.dbo.loadcontroltable LIMIT 1000")
display(df)

StatementMeta(, 8dd3a5dc-5f8f-4a13-897e-dd4edfc4d97b, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 08896e02-c73f-4fed-b901-915c912c7aa4)

In [ ]:
# df = spark.read.option("header", "True").option("delimiter", ",").option("inferSchema", True).csv("Files/output/circuits_data_quality_dup.csv/")
# display(df)